In [1]:
import pandas as pd
import OpenDartReader
import requests as rq
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
from tqdm import tqdm
import time
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)

def oder_backlog(corp_name):

    print(corp_name)
    #기간
    lists = dart.list(corp_name, start='2021-01-01', end ='2024-12-30', kind='A') 
    
    lists['공시'] = lists['report_nm'].str.split(' ').str[1]
    # lists['report_nm'] = lists['report_nm'].str.contains('사업')


    rcp_no = []
    for i in range(0,len(lists)):
        rcp = lists['rcept_no'][i]
        rcp_no.append(rcp)  
        

    df = pd.DataFrame()
    comment = ['직원','직 원']

    for j in tqdm(range(0,len(rcp_no))):
        try:    
            files = dart.sub_docs(rcp_no[j])
            # files = dart.sub_docs('20231114000825')
            content = [] #

            data = files[files['title'].str.contains('1. 임원')]
            
            a = data['url'].values[0]
            res = rq.get(a,headers=headers)
            soup=BeautifulSoup(res.text,'html.parser')
            temp = soup.select('body > table')
                    
            for v in temp:
                # print(v)
                text = v.get_text()
                if '직원' in text or '직 원' in text : 
                    # print(text)
                    content.append(v)
            if '급여' in content[0].text.strip():
                table = parser.make2d(content[0]) #표로 만들어줌
            elif '급여' in content[1].text.strip():
                table = parser.make2d(content[1]) #표로 만들어줌
            elif '급여' in content[2].text.strip():
                table = parser.make2d(content[2]) #표로 만들어줌
            elif '급여' in content[3].text.strip():
                table = parser.make2d(content[3]) #표로 만들어줌
            adata = pd.DataFrame(table) # 필요한부분만 불러옴
            adata.columns = adata.iloc[2]
            adata = adata.iloc[-1:,:]
            adata = adata[['합 계']]

            
            adata.insert(0,'날짜',lists['공시'][j])
            adata['날짜'] = adata['날짜'].str.replace('(','',regex=True)
            adata['날짜'] = adata['날짜'].str.replace(')','',regex=True)
            adata.insert(0,'기업',corp_name)
            
                    
            df = pd.concat([df,adata],ignore_index=True)
            time.sleep(1)
            
        except Exception as e:
            print(f'{corp_name}',e)
        pass
    return df

# ddf = pd.read_clipboard()
# ddf['종목코드'] = ddf['종목코드'].apply(lambda x: f'{x:06d}')

# # '종목코드' 열을 리스트로 변환
# lst = ddf['종목코드'].tolist()
lst = ['103590']



# corp_name = oder_backlog(lst)
# print(corp_name)


dd = pd.DataFrame()
for i in lst:
    try:
        corp_name = oder_backlog(i)
        dd = pd.concat([dd,corp_name])
    except Exception as e:
        print(e)
dd.to_clipboard()
# dd.to_excel(f'직원수_{today}.xlsx')


103590


100%|██████████| 16/16 [00:19<00:00,  1.19s/it]
